In [18]:
import pandas as pd
import math
from collections import Counter
import numpy as np
import pickle
from sklearn import metrics
import re

In [19]:
df = pd.read_csv('./data/dataset.csv')
df = df.loc[:15000]
df.shape

(15001, 2)

In [20]:
first_col = df.iloc[1:, 0]
second_col = df.iloc[1:, 1]
second_col = second_col.fillna(0)

In [21]:
stop_words_file = open("./data/stopwords.txt","r",encoding="utf-8")
stop_words = stop_words_file.read()
stop_words = stop_words.split("\n")

In [22]:
#data cleaning method
def data_cleaning(string):
    text = re.sub('\,|\@|\-|\"|\'| \)|\(|\)| \{| \}| \[| \]|!|‘|’|“|”| \:-|\?|।|/|\—|\०|\१|\२|\३|\४|\५|\६|\७|\८|\९|[0-9]', '', string)
    return text

def stop_word_remove(array_element):
    array_element_set = set(array_element)
    final_list = list(array_element_set.difference(stop_words))
    return final_list

In [23]:

def tokenize():
    data_with_split = []
    for data in first_col:
        return_string = data_cleaning(data)
        each_docs = return_string.split(" ")
        string_after_remove_word=stop_word_remove(each_docs)
        
        data_with_split.append(each_docs)
    return data_with_split  # it returns arr of each docs with spleted words



corpus = tokenize()

In [24]:
class TFIDFVectorizer:
    def __init__(self):
        self.idf = None
        self.vocabulary = set()
        
        # Build vocabulary
        for document in corpus:
            self.vocabulary.update(document)
        self.vocabulary = list(self.vocabulary)

    def get_tif_idf_info(self,words,sentence_feature):
        tf_idf_info = {}
        for word in words:
            index = self.vocabulary.index(word)
            tf_idf_info[word]=sentence_feature[0][index]
        return tf_idf_info
    
    def fit_transform(self, corpus):
        # Calculate IDF
        idf = {}
        N = len(corpus)
        for term in self.vocabulary:
            df = sum(1 for document in corpus if term in document)
            idf[term] = math.log(N / (1 + df))

        # Transform documents to TF-IDF representation
        tfidf_matrix = np.zeros((len(corpus), len(self.vocabulary)))
        for i, document in enumerate(corpus):
            tf = Counter(document)
            total_terms = len(document)
            for j, term in enumerate(self.vocabulary):
                if total_terms != 0:
                    tfidf_matrix[i, j] = (tf.get(term, 0) / total_terms) * idf[term]
                else:
                    tfidf_matrix[i, j] = 0  # Set TF-IDF to 0 if total_terms is 0

        self.idf = idf
        return tfidf_matrix

    def transform(self, corpus):
        tfidf_matrix = np.zeros((len(corpus), len(self.vocabulary)))
        for i, document in enumerate(corpus):
            tf = Counter(document)
            total_terms = len(document)
            for j, term in enumerate(self.vocabulary):
                if total_terms != 0:
                    tfidf_matrix[i, j] = (tf.get(term, 0) / total_terms) * self.idf.get(term, 0)
                else:
                    tfidf_matrix[i, j] = 0  # Set TF-IDF to 0 if total_terms is 0
        return tfidf_matrix

# Create TFIDFVectorizer instance
tfidf_vectorizer = TFIDFVectorizer()

# Fit and transform corpus
features = tfidf_vectorizer.fit_transform(corpus)

vectorizer_data = open("./webapp/model/vectorizer.pkl", "wb")
pickle.dump(tfidf_vectorizer, vectorizer_data)
vectorizer_data.close()

with open('./webapp/model/vectorizer.pkl', 'rb') as tfidf:
    vectorizer = pickle.load(tfidf)

In [25]:
from sklearn.model_selection import train_test_split
x=features
y=second_col
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.2,random_state=100)


In [26]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()  
TrainData = naive_bayes.fit(train_x, train_y)

classifier_data = open("./webapp/model/classifier.pkl", "wb")
pickle.dump(naive_bayes, classifier_data)
classifier_data.close()

In [27]:
with open('./webapp/model/classifier.pkl', 'rb') as pickle_saved_data:
    unpickled_data = pickle.load(pickle_saved_data)



prediction = unpickled_data.predict(test_x)

In [30]:
def calculate_performance_metrics(true_labels, predicted_labels):
    precision = metrics.precision_score(true_labels, predicted_labels, average='weighted')
    recall = metrics.recall_score(true_labels, predicted_labels, average='weighted')
    accuracy = metrics.accuracy_score(true_labels, predicted_labels)
    f1_score = metrics.f1_score(true_labels, predicted_labels, average='weighted')
    
    return precision, recall, accuracy, f1_score

# Example usage:
precision, recall, accuracy, f1_score = calculate_performance_metrics(test_y, prediction)

metrices = [precision, recall, accuracy, f1_score]
print(metrices)
with open("./webapp/model/metrices.pkl", "wb") as metrices_file:
    pickle.dump(metrices, metrices_file)

with open('./webapp/model/metrices.pkl', 'rb') as metrics_file:
    metrics = pickle.load(metrics_file)

print("Precision:",metrics[0])   
print("Recall:",metrics[1])  
print("F1 Score:",metrics[2])  
print("Accuracy:",metrics[3])   



AttributeError: 'list' object has no attribute 'precision_score'

In [29]:
def predict_sentiment(sentence):
    # Preprocess the input sentence
    cleaned_sentence = data_cleaning(sentence)
    tokenized_sentence = cleaned_sentence.split()
    stop_word_removed_sentence = stop_word_remove(tokenized_sentence)
    
    # Transform the preprocessed sentence using TF-IDF vectorizer
    sentence_features = vectorizer.transform([stop_word_removed_sentence])

    # Use the trained classifier to predict the sentiment label
    predicted_label = unpickled_data.predict(sentence_features)

    return predicted_label[0]  # Return the predicted sentiment label

# Example usage:
sentence = "आज म धेरै खुसी छु।"
predicted_sentiment = predict_sentiment(sentence)
print("Input sentence:", sentence)
print("Predicted Sentiment Label:", predicted_sentiment)


Input sentence: आज म धेरै खुसी छु।
Predicted Sentiment Label: 1
